In [34]:
## 手动搭建神经网络

In [35]:
# 创建过程
import tensorflow as tf
import numpy as np
import pandas as pd
from  sklearn.model_selection import train_test_split

In [36]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [37]:
# 使用 placeholder 来表示输入输出
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [38]:
# 创建层
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1]) #获取输入特征数 784， 300 ，100
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [39]:
# 创建神经网络
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

In [40]:
# 使用内置创建层函数 创建神经网络
from tensorflow.contrib.layers import fully_connected

In [41]:
with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")  #默认激活函数为relu
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn=None)

ValueError: Variable hidden1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 246, in variable
    use_resource=use_resource)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 183, in func_with_args
    return func(*args, **current_args)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 291, in model_variable
    use_resource=use_resource)


In [ ]:
# 下面定义损失函数
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
# 有模型，有损失函数， 需要定义 优化器(GradientDescentOptimizer), 最小化损失函数。
learing_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learing_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
# 创建过程的最后一步，评价模型，简单还是用精确度来衡量
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
# 创建节点， 初始化所有变量， 创建Saver保存模型和参数
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## 执行阶段

In [ ]:
# 加载数据
train_df = pd.read_csv("./datasets/train.csv")
test_df = pd.read_csv("./datasets/test.csv")
combine = [train_df, test_df]

In [ ]:
# 定义执行步数和 小批量更新的实例数
n_epochs = 50
batch_size = 50

In [ ]:
train_df.head()

In [ ]:
X_train = train_df.drop(["label"], axis=1)

y_train = train_df["label"]


In [ ]:
from sklearn.preprocessing import StandardScaler
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(test_df)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape

In [ ]:
    m, n = X_train.shape

In [ ]:
# 训练模型
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(m // batch_size):
            X_batch = X_train[iteration: iteration + batch_size]
            y_batch = y_train.iloc[iteration: iteration + batch_size].values
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, "train accurancy:", acc_train)
    save_path = saver.save(sess, "./model/final.ckpt")

In [ ]:
# 重载模型， 进行预测
with tf.Session() as sess:
    saver.restore(sess, "./model/final.ckpt")
    Z = logits.eval(feed_dict={X:X_test_scaled})
    y_pred = np.argmax(Z, axis=1)

In [ ]:

# 保存结果到文件
# ImageId = np.arange(1, 28001)
# result = pd.DataFrame({
#     "ImageId": ImageId,
#     "Label": y_pred
# })
# result.to_csv("result.csv", index=False)